In [ ]:
from instagrapi import Client
from instagrapi.exceptions import UserNotFound
import configparser, time, shutil, os, re, unicodedata

config = configparser.ConfigParser()
config.read("config.ini")

USERNAME = config["instagram"]["username"]
PASSWORD = config["instagram"]["password"]

cl = Client()
cl.login(USERNAME, PASSWORD)

True

In [ ]:
# -*- coding:utf-8 -*-
def get_accounts_num(target_account, config):

    # 정규식으로 계정명과 숫자 추출
    match = re.match(r"(https?://www\.instagram\.com/)?([\w\.]+)(?:\s*//\s*(\d+))?", target_account)

    target_account = match.group(2)  # 계정명
    target_post_count = int(match.group(3)) if match.group(3) else int(config["instagram"]["DEFAULT_NUM"])

    return target_account, target_post_count


config = configparser.ConfigParser()
config.read("config.ini")

ACCOUNTS_LIST = config["instagram"]["accounts_list"].splitlines()
ACCOUNTS_LIST = [url.strip() for url in ACCOUNTS_LIST if url and url.strip()]

# ☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★
for target_account in ACCOUNTS_LIST:

    target_account, target_post_count = get_accounts_num(target_account, config)

    try:
        user_info = cl.user_info_by_username(target_account)
    except UserNotFound:
        print(f"User not found. Please check the username. -> {target_account}")
        continue
    user_id, user_name = user_info.pk, user_info.full_name

    download_folder = f"{user_name} (insta_{target_account})"
    download_folder = os.path.join(config["instagram"]["default_path"], download_folder)
    download_folder = re.sub(r'[*?"<>|]', "", download_folder.replace("|", "｜"))
    download_folder = unicodedata.normalize("NFC", download_folder)
    print(download_folder)

    if not os.path.exists(download_folder):  # 폴더 생성
        os.makedirs(download_folder)

    medias = cl.user_medias(user_id, target_post_count)  # 게시글 정보 수집

    for i, media in enumerate(medias):
        print(f"\r{user_name} (insta_{target_account}) \t {i+1}/{target_post_count} - {(i+1)/target_post_count*100:.1f}%", end="")
        download_methods = {1: cl.photo_download, 2: cl.video_download, 8: cl.album_download}  # Photo  # Video  # Album
        return_paths = download_methods.get(media.media_type, lambda *_: None)(media.pk, download_folder)
        if not isinstance(return_paths, list):
            return_paths = [return_paths]

        for ii, path in enumerate(return_paths):
            prefix, ext = str(path).rsplit("\\", 1)[0], str(path).rsplit(".", 1)[1]
            new_path = os.path.join(prefix, (f"[{media.taken_at.date()}] {media.code}_p{ii}.{ext}"))
            shutil.move(str(path), new_path)
    else:
        print("")